## Importing libraries

In [93]:
import sys
import requests
import os
gp = '/home/jupyter-gabriel/'#'I:/My Drive/'
sys.path.append(gp + "projects/suyana")
from scripts.utils.functions import *

In [94]:
# Data primary path
dpp = 'suyana/'
# Data inputs path
dip = 'suyana/inputs/'
# Data features path
dfp = 'suyana/features/'
# Data outputs path
dop = 'suyana/outputs/'
# Data reports path
drp = 'suyana/reports/'
# Data models path
dmp = 'suyana/models/'
# Data displays path
ddp = 'suyana/displays/'
# Data geometries path
dgp = 'suyana/geometries/'

In [95]:
path = dip
files = os.listdir(path)
files = [f for f in files if f.startswith("drought_canton_processed")]

id_canton = [files[i].split("_")[1].split(".")[0] for i in range(len(files))]

In [105]:
list_dates = []
list_swc = []
list_id = []
data = pd.DataFrame()
for file in files:
    id_canton = file.split("_")[1].split(".")[0]
    netcdf_file = xr.open_dataset(path + file)
    netcdf_file = netcdf_file.mean(dim=["lat", "lon"])
    
    data_aux = pd.DataFrame({
        "date": netcdf_file.time.values,
        "swc": netcdf_file.swc_adjusted.values
    })
    data_aux["idCanton"] = id_canton

    data = pd.concat([data, data_aux], axis=0)

## Exploratory Data Analysis

In [106]:
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month

In [107]:
netcdf_file.variables

Frozen({'swc': <xarray.Variable (time: 8175)> Size: 65kB
array([       nan, 0.2963148 , 0.2963148 , ..., 0.2574205 , 0.26587033,
       0.26587033]), 'swc_adjusted': <xarray.Variable (time: 8175)> Size: 65kB
array([       nan, 0.30202562, 0.31432067, ..., 0.2574205 , 0.26587033,
       0.26587033]), 'time': <xarray.IndexVariable 'time' (time: 8175)> Size: 65kB
array(['2002-06-15T00:00:00.000000000', '2002-06-16T00:00:00.000000000',
       '2002-06-17T00:00:00.000000000', ..., '2024-10-29T00:00:00.000000000',
       '2024-10-30T00:00:00.000000000', '2024-10-31T00:00:00.000000000'],
      dtype='datetime64[ns]'), 'week': <xarray.Variable (time: 8175)> Size: 65kB
[8175 values with dtype=int64]})

In [108]:
data_summary = data.groupby(["idCanton", "year"]).size().reset_index()

In [109]:
## CANTIDAD DE OBSERVACIONES POR AÑO
data_summary.pivot(index="idCanton", columns="year", values=0)

year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
idCanton,,,,,,,,,,,,,,,,,,,,,,,
canton,1800,3285,3294,3285,3285,3285,3294,3285,3285,3285,3294,3285,3285,3285,3294,3285,3285,3285,3294,3285,3285,3285,2745


In [70]:
## DUPLICADOS
data[data.year==2012].groupby(["date", "idCanton"]).agg(n=("idCanton","count")).reset_index().query('n>1')

,date,idCanton,n
3485,2012-07-24,43,2
3486,2012-07-24,45,2
3487,2012-07-24,49,2
3488,2012-07-24,50,2
3489,2012-07-24,51,2
3490,2012-07-24,52,2
3491,2012-07-24,53,2
3492,2012-07-24,54,2
3493,2012-07-24,55,2
3494,2012-07-24,56,2


In [110]:
n_anios = len(data['year'].unique())
min_year = min(data['year'])
max_year = max(data['year'])
print(f'Cantidad de años: {n_anios} desde {min_year} hasta {max_year}')
n_dates = len(data['date'].unique())
min_date = min(data['date'])
max_date = max(data['date'])
print(f'Cantidad de días: {n_dates} desde {min_date} hasta {max_date}')

Cantidad de años: 23 desde 2002 hasta 2024
Cantidad de días: 8175 desde 2002-06-15 00:00:00 hasta 2024-10-31 00:00:00


# Get canton id for yields panel data

In [111]:
# Data from gpkg cantones bolivia
df_cantones = gpd.read_file(dgp + 'drought_canton_polygons.gpkg')

In [120]:
# data_folder = Path("../Data/data_output_rendimiento")
# filename = "df_rendimiento_flagged_ONI.csv"
data_folder = dip
filename = "drought_yields_panel.csv"
df_yields = pd.read_csv(data_folder + filename)
if set(['flag','notes', 'Unnamed: 10'] ).issubset(df_yields.columns):
    df_yields.drop(['flag','notes', 'Unnamed: 10'], axis=1, inplace = True)

In [121]:
df_aux = df_yields[['longitude', 'latitude']].drop_duplicates()
df_aux['coords'] = list(zip(df_aux['longitude'], df_aux['latitude']))

df_coords = gpd.GeoDataFrame({
    'longitude': list(df_aux['longitude']),
    'latitude': list(df_aux['latitude']),
    'geometry': [Point(p) for p in df_aux['coords']]
}, crs = df_cantones.crs)

df_join = df_coords.sjoin(
    df_cantones,
    how = 'left',
    predicate = 'within'
)

In [122]:
df_yields_final = df_yields.merge(
    df_join[['longitude', 'latitude', 'id_canton', 'canton_original', 'provincia', 'municipio', 'departamen']],
    how = 'left',
    on = ['longitude', 'latitude'],
)

In [123]:
df_yields_final.id_canton.unique()

array([189,  56,  52,  49,  57, 195,  43,  54, 209])

## Clean data from yields panel

In [124]:
def group_categories_MT(df_orig, group_categories):
    """
    Groups the categories of a categorical variable in a DataFrame.

    Parameters:
        df_orig (pd.DataFrame): DataFrame containing the client variables.
        group_categories (dict): Dictionary that contains the categories to be grouped.
                                 The keys are the column names, and the values are dictionaries
                                 where the keys are the new categories and the values are lists
                                 of the original categories to be grouped under the new category.

    Returns:
        pd.DataFrame: Modified DataFrame with grouped category columns.
    """
    df = df_orig.copy()
    
    features_group = group_categories["categories"]

    for column in features_group:
        for new_category, original_categories in features_group[column].items():
            df[column] = df[column].replace(original_categories, new_category)

    return df

In [125]:
def modify_categories(
    df_orig: pd.DataFrame(), params_modificacion_categorias
) -> pd.DataFrame():
    """Modify categories in the dataset based on specified values.

    Args:
        df_orig (pd.DataFrame): The complete dataset with all variables.
        params_modificacion_categorias (dict): A dictionary containing the column and the desired categories for that column.
            Other categories not specified will be replaced with 'OTRO'.

    Returns:
        pd.DataFrame: The modified dataset with updated categories.

    Example:
        >>> df = modify_categories(df, params_modificacion_categorias)
    """

    df = df_orig.copy()

    # Parameters
    variables_valores = params_modificacion_categorias["variables_valores"]

    for col in variables_valores:
        print(col)
        df[col] = df[col].apply(
            lambda x: x if x in variables_valores[col] else "OTRO"
        )

    return df


In [126]:
def treat_nans(df_orig, params):
    """
    Treats missing values in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing the client variables.
        params (dict): Dictionary containing the parameters to treat missing values.

    Returns:
        pd.DataFrame: Modified DataFrame with treated missing values.
    """
    df = df_orig.copy()
    # Parameters
    missing_vars = params["missing_vars"]
    method = params["method"]
    group_vars = params["group_vars"]

    if set(group_vars).intersection(set(missing_vars)) != set():
        print("Error: group_vars and missing_vars should not have common variables.")
        return
    # Treat missing values
    if method == "remove":
        df = df.dropna(subset=missing_vars)
    elif method == "impute":
        for var in missing_vars:
            if df[var].dtype == "object":
                df[var] = df[var].fillna("MISSING")
            elif len(group_vars) == 0:
                df[var] = df[var].fillna(df[var].mean())
            else:
                df[var] = df.groupby(group_vars)[var].transform(
                    lambda x: x.fillna(x.mean())
                )

    return df

In [127]:
def apply_log_to_var(df_orig, var, fill_value="mean"):
    """
    Apply the natural logarithm to the specified variables in the DataFrame.

    Parameters:
        df_orig (pd.DataFrame): DataFrame containing the client variables.
        params (dict): Dictionary containing the parameters to apply the natural logarithm.

    Returns:
        pd.DataFrame: Modified DataFrame with the natural logarithm applied to the specified variables.
    """
    df = df_orig.copy()
       
    # Apply the natural logarithm
    min_value = df.loc[df[var] > 0, var].min()
    df[var] = np.where(df[var] <= 0, min_value*0.9, df[var])

    df["log_"+var] = np.log(df[var])
    df["log_"+var] = np.where(
        df["log_"+var].isin([np.inf, -np.inf]), 
        np.nan, 
        df["log_"+var]
    )
    if fill_value == "mean":
        df["log_"+var] = df["log_"+var].fillna(df["log_"+var].mean())
    else:
        df["log_"+var] = df["log_"+var].fillna(fill_value)

    return df

In [128]:
params_group_categ = {
    'categories': {
        'culture': {
            #'GIRASOL': ['Girasol'],
            'MAIZ': ['MAIZ', 'MAIZ ENSILAJE'],
            'TRIGO': ['TRIGO'],
            'SOYA': ['SOYA'],
            'SORGO': ['SORGO', 'SORGO DOBLE PROPOSITO', 'SORGO DOBLE PROPOSITO ENSILAJE', 'SORGO FORRAJERO'],
        }
    }
}
params_modif_categ = {
    'variables_valores': {
        'culture': ['MAIZ', 'TRIGO', 'SOYA', 'SORGO']
    }
}
params_nans = {
    'missing_vars': ['rend_ton_ha'], #, 'ndvi', 'ndpi', 'pp', 'vhi', 'anomaly_sum', 'swc_sum'
    'method': 'impute',
    'group_vars': []
}

In [129]:
df_yields_final['culture_orig'] = df_yields_final['culture']
df_yields_final = group_categories_MT(df_yields_final, params_group_categ)
df_yields_final = modify_categories(df_yields_final, params_modif_categ)
df_yields_final = treat_nans(df_yields_final, params_nans)

culture


In [130]:
df_yields_final['culture'].value_counts()

culture
SOYA     433
SORGO    116
TRIGO    101
MAIZ      59
OTRO      51
Name: count, dtype: int64

In [ ]:
df_yields_final.to_csv(dip + 'drought_yields_panel_id.csv', index=False)

In [131]:
df_yields_final = pd.read_csv(dip+'drought_yields_panel_id.csv')

In [148]:
df_yields_final = pd.read_csv(dip+'drought_yields_panel.csv')
df_yields_final

,producer_id,zone,lot,plot,orig_crop,variety,seedtime,harvest_date,flag,Unnamed: 10,show_furrow,n_plants_m,n_plants_ha,date_beginning_flowering,mature_maturation_date,plant_height,fertitilized,type_of_fertilizer,dose_kg_ha,rend_ton_ha,cond_climaticas_de_la_campana,producer_handling,notes,id_lot,cultivation_id,...,pp_anomaly_seed_date_minus_2_month,vhi_anomaly_seed_date_minus_2_month,ndvi_anomaly_flowering_date,ndpi_anomaly_flowering_date,pp_anomaly_flowering_date,vhi_anomaly_flowering_date,ndvi_flowering_date,ndpi_flowering_date,pp_flowering_date,vhi_flowering_date,ndvi_anomaly_flowering_date_minus_1_month,ndpi_anomaly_flowering_date_minus_1_month,pp_anomaly_flowering_date_minus_1_month,vhi_anomaly_flowering_date_minus_1_month,ndvi_anomaly_flowering_date_minus_2_month,ndpi_anomaly_flowering_date_minus_2_month,pp_anomaly_flowering_date_minus_2_month,vhi_anomaly_flowering_date_minus_2_month,Month,Year,ENSO,pixel_id,anomaly_sum,swc_sum,climatology_sum
0,16,Norte,H-2,Las Londras,Sorgo,Timbo,NaN,2016-09-13,False,NaN,0.45,11.0,244444.440,45,110,1.1,MAP,11-52-0,100,1.000000,NaN,NaN,NaN,67,2,...,-147.570695,-931.913044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,2016.0,ninia,0.0,0.000000,0.0,0.000000
1,14,Este,B04,Europa,Maiz,Maiz 355,2013-03-10,2013-08-20,True,NaN,0.42,2.4,57142.857,5/14/2013,7/13/2013,2.1,0,0,0,1.994742,NaN,NaN,NaN,45,1,...,-318.040930,-1266.048243,-573.160520,1.054872,7.770585,-25.339241,6219.065355,1.120897,18.280309,27.384402,-22986.655761,-4.350322,-99.537446,-883.337060,-36100.439166,1.791396,-109.046194,-1606.263388,8.0,2013.0,NaN,0.0,10141.265152,25988.0,15846.734848
2,14,Este,A09,Europa,Sorgo,DAS-5000,2013-03-19,2013-07-24,True,NaN,0.42,9.0,214285.710,5/3/2013,7/7/2013,1.1,0,0,0,2.323631,NaN,NaN,NaN,43,2,...,-252.626529,-1279.054223,1549.200627,0.287106,0.638863,-27.786188,6749.498087,0.222299,9.864622,24.390516,22223.058567,11.863377,107.183678,-914.857636,-14995.596168,0.469207,-157.261620,-1528.424227,7.0,2013.0,NaN,0.0,8117.090909,20801.0,12683.909091
3,12,Este,15-N,AgroOriente,Trigo,A9829R,2013-03-31,2013-08-09,True,NaN,0.17,50.0,2941176.500,5/9/2013,7/13/2013,NaN,0,0,0,NaN,Malo,Regular,NaN,27,5,...,-503.627799,-390.735870,1530.561548,-0.418810,-6.299874,-9.022686,7196.695431,-0.459921,4.003364,38.563123,7854.427149,4.080776,32.109304,-470.953482,-18272.315065,1.485035,-102.000497,-793.850622,8.0,2013.0,NaN,0.0,8249.757576,20998.0,12748.242424
4,12,Este,14-N,AgroOriente,Trigo,A9829R,2013-04-01,2013-08-17,True,NaN,0.17,50.0,2941176.500,5/10/2013,7/14/2013,NaN,0,0,0,NaN,Malo,Regular,NaN,25,5,...,-533.974944,-394.039708,1967.987610,-0.393664,-6.289624,-8.906942,7336.117424,-0.402046,4.600607,38.165354,15752.476641,1.861606,2.389222,-461.689851,-10461.061711,1.004339,-104.588432,-797.567208,8.0,2013.0,NaN,0.0,8572.257576,22023.0,13450.742424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,14,Este,F11,Europa,Barbecho,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.000000,NaN,NaN,NaN,50,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000
756,14,Este,D02,Europa,Potrero,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.000000,NaN,NaN,NaN,48,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000
757,14,Este,D02,Europa,Potrero,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.000000,NaN,NaN,NaN,48,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000
758,14,Este,D08,Europa,Potrero,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0.000000,NaN,NaN,NaN,49,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.000000


## CDF Matching function example

In [166]:
x = np.array([5,4,4,3,2,np.nan])
y = np.array([2,1,1,4,np.nan, np.nan])

x_valid = x[~np.isnan(x)]
y_valid = y[~np.isnan(y)]

x_cdf = np.sort(x_valid)
y_cdf = np.sort(y_valid)

x_indices = np.searchsorted(x_cdf, x, side="right")
y_quantiles = x_indices / len(x_cdf)  # Quantiles of x in the valid x range
y_indices = np.floor(y_quantiles * (len(y_cdf)-0)).astype(
    int
)  # Corresponding indices in y

adjusted_x = np.take(y_cdf, y_indices, mode="clip")

In [167]:
adjusted_x

array([4., 4., 4., 1., 1., 4.])

In [168]:
y_indices

array([4, 3, 3, 1, 0, 4])

In [169]:
y_cdf

array([1., 1., 2., 4.])

In [170]:
y_quantiles

array([1. , 0.8, 0.8, 0.4, 0.2, 1. ])

# Toy data for xarray

In [103]:
ds_toy = xr.Dataset(
    data_vars = dict(
        temp = (['lon', 'lat','time'], np.ones((5,5,2))),
    ),
    coords = dict(
        lon = np.arange(1,6),
        lat = np.arange(1,6),
        time = np.arange(1,3)
    )
)
ds_toy

<xarray.Dataset> Size: 496B
Dimensions:  (lon: 5, lat: 5, time: 2)
Coordinates:
  * lon      (lon) int64 40B 1 2 3 4 5
  * lat      (lat) int64 40B 1 2 3 4 5
  * time     (time) int64 16B 1 2
Data variables:
    temp     (lon, lat, time) float64 400B 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0

In [104]:
ds_toy2 = ds_toy.coarsen({'lon':3, 'lat': 3}, boundary='pad').sum()

ds_toy2.to_dataframe().reset_index()

,lon,lat,time,temp
0,2.0,2.0,1,9.0
1,2.0,2.0,2,9.0
2,2.0,4.5,1,6.0
3,2.0,4.5,2,6.0
4,4.5,2.0,1,6.0
5,4.5,2.0,2,6.0
6,4.5,4.5,1,4.0
7,4.5,4.5,2,4.0


In [45]:
ds_toy.coarsen(lon=len(ds_toy.lon)-len(ds_toy.lon)//2, lat=len(ds_toy.lat)-len(ds_toy.lat)//2, boundary = 'trim').sum()

<xarray.Dataset> Size: 112B
Dimensions:  (lon: 2, lat: 2, time: 2)
Coordinates:
  * lon      (lon) float64 16B 1.5 3.5
  * lat      (lat) float64 16B 1.5 3.5
  * time     (time) int64 16B 1 2
Data variables:
    temp     (lon, lat, time) float64 64B 4.0 4.0 4.0 4.0 4.0 4.0 4.0 4.0